In [ ]:
import pickle
import numpy as np
import tensorflow as tf
import os

# Get the data
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


def get_data(file_path, first_class, second_class):
    
    unpickled_file = unpickle(file_path)
    inputs = unpickled_file[b'data']
    labels = unpickled_file[b'labels']
    
    # Extract inputs and labels only in the first and the second class
    labels = np.array(labels)
    new_inputs = inputs[np.array([i for i in range(len(labels)) if labels[i] == first_class or labels[i] == second_class]),] 
    new_labels = labels[np.array([i for i in range(len(labels)) if labels[i] == first_class or labels[i] == second_class])]
    
    # Change the labels 
    new_labels[np.array([i for i in range(len(new_labels)) if new_labels[i] == first_class])] = 0
    new_labels[np.array([i for i in range(len(new_labels)) if new_labels[i] == second_class])] = 1
    
    # Normalize the inputs
    new_inputs = new_inputs/255.0
    new_inputs = new_inputs.astype(np.float32)
    
    # Onehot labels
    new_labels = tf.one_hot(new_labels, depth =2)
    
    return (new_inputs, new_labels)


class MLP(tf.keras.Model):
    def __init__(self, pixel_size):
        
        self.pixel_size = pixel_size
        self.hidden_size = 128
        self.batch_size = 64
        
        super(MLP, self).__init__()
        self.W1 = tf.Variable(tf.random.truncated_normal([self.pixel_size, self.hidden_size], stddev = 0.1))
        self.b1 = tf.Variable(tf.random.truncated_normal([self.hidden_size], stddev = 0.1))
        self.W2 = tf.Variable(tf.random.truncated_normal([self.hidden_size, 2], stddev = 0.1))
        self.b2 = tf.Variable(tf.random.truncated_normal([2], stddev = 0.1))
        
    def call(self, inputs):
        output1 = tf.add(tf.linalg.matmul(inputs, self.W1), self.b1)
        final_output1 = tf.nn.relu(output1)
        logits = tf.add(tf.linalg.matmul(final_output1, self.W2), self.b2)
        
        return logits
    
    def loss(self, logits, labels):
        loss = tf.nn.softmax_cross_entropy_with_logits(labels = labels, logits = logits)
        return tf.reduce_mean(loss)
    
    def accuracy(self, logits, labels,):
        correct_predictions = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
        return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    
    
def train(model, train_inputs, train_labels):
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-3)
    
    for x in range(0, train_inputs.shape[0], model.batch_size):
        batch_inputs = train_inputs[x: x + model.batch_size, :]
        batch_labels = train_labels[x: x + model.batch_size]
        
        # Optimize gradients
        with tf.GradientTape() as tape:
            predictions = model.call(batch_inputs)
            loss = model.loss(predictions, batch_labels)
      
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        
def test(model, test_inputs, test_labels):
    acc = []
    for x in range(0, test_inputs.shape[0], model.batch_size):
        batch_inputs = test_inputs[x: x + model.batch_size, :]
        batch_labels = test_labels[x: x + model.batch_size]
        
        predictions = model.call(batch_inputs)
        acc.append(model.accuracy(predictions, batch_labels))
    
    return np.mean(acc)

In [ ]:
# Get the data
(inputs, labels) = get_data('train', first_class=3, second_class=5)
(test_inputs, test_labels) = get_data('test', first_class=3, second_class=5)
model = MLP(3072)

epochs = 1
for i in range(epochs):
    train(model, inputs, labels)
    test_acc = test(model, test_inputs, test_labels)
    print(test_acc)

print("Accuracy on test set after {} epochs steps: {}".format(epochs, test_acc))

#print(model.W1)

W1 = model.W1
#b1 = model.b1
W2 = model.W2
#b2 = model.b2

# r = tf.constant(tf.random.truncated_normal(shape=[3072],stddev=0.001))
x = tf.reshape(inputs[0,],[-1,3072])
k_pred = tf.argmax(model(x), 1).numpy()[0]
k_true = tf.argmax(model(tf.reshape(inputs[0,],[-1,3072])), 1).numpy()[0]
r_out = tf.constant(0,shape=[3072],dtype=tf.float32)
while k_pred == k_true:
    gradient = tf.matmul(W1,W2)[:,1-k_true] - tf.matmul(W1,W2)[:,k_true]
    f_k = model(x)[:,1-k_true] - model(x)[:,k_true]
    r = tf.abs(f_k) / tf.square(tf.norm(gradient)) * gradient
    x += r
    k_pred = tf.argmax(model(x), 1).numpy()[0]
    r_out += r
print(r_out)



In [ ]:
tf.reshape(x,[3,32,32])

In [ ]:
from matplotlib import pyplot as plt

# original image
plt.imshow(tf.reshape(inputs[0,],[3,32,32]).numpy().transpose(1,2,0))
plt.show()
# after adding noise
plt.imshow(tf.reshape(x,[3,32,32]).numpy().transpose(1,2,0))
plt.show()